In [5]:
import rasterio
import pandas as pd
import numpy as np
import pyproj
import os

In [6]:
points = pd.read_csv('../lora/data/shp/patch_coords.csv')

source_crs = pyproj.CRS("EPSG:4326")  # WGS 84
target_crs = pyproj.CRS("EPSG:25833")  # UTM zone 33N

transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

points["x"], points["y"] = zip(*points.apply(lambda row: transformer.transform(row["lon"], row["lat"]), axis=1))

In [7]:
#only actual point value used
topo_points = None

in_path = '../rasters/topo_tifs'
out_path = 'merge_ready/'

for filename in os.listdir(in_path):
    pixel_values = []
    with rasterio.open(os.path.join(in_path, filename)) as src:
        # Extract pixel values
        for _, point in points.iterrows():
            for band in range(1, src.count+1):
                row, col = src.index(point['x'], point['y'])
                value = src.read(band)[row, col]
                variable = filename[:-4]
                pixel_values.append([point['patch'], point['x'], point['y'], variable, value])
        # Organize data
        columns = ['patch', 'x', 'y', 'variable', 'pixel_value']
        pixel_df = pd.DataFrame(pixel_values, columns=columns)
        if topo_points is None:
            topo_points = pixel_df
        else:
            topo_points = pd.concat([topo_points, pixel_df])

#Pivot columns to wider
topo_points_single = topo_points.pivot(index='patch', columns='variable', values='pixel_value')
topo_points_single.reset_index(inplace=True)

# output as csv
topo_points_single.to_csv(os.path.join(out_path, "topo_1_mr.csv"), index = False)

In [8]:
#neighborhood mean calculated per point
topo_points = None

in_path = '../rasters/topo_tifs'
out_path = 'merge_ready/'

for filename in os.listdir(in_path):
    pixel_values = []
    with rasterio.open(os.path.join(in_path, filename)) as src:
        # Extract pixel values
        for _, point in points.iterrows():
            for band in range(1, src.count+1):
                row, col = src.index(point['x'], point['y'])
                neighborhood = src.read(band)[max(row-1,0):row+2, max(col-1,0):col+2]
                mean_value = neighborhood.mean()
                variable = filename[:-4]
                pixel_values.append([point['patch'], point['x'], point['y'], variable, mean_value])
        
        # Organize data
        columns = ['patch', 'x', 'y', 'variable', 'pixel_value']
        pixel_df = pd.DataFrame(pixel_values, columns=columns)
        if topo_points is None:
            topo_points = pixel_df
        else:
            topo_points = pd.concat([topo_points, pixel_df])

#Pivot columns to wider
topo_points_neighbor = topo_points.pivot(index='patch', columns='variable', values='pixel_value')
topo_points_neighbor.reset_index(inplace=True)

# output as csv
topo_points_single.to_csv(os.path.join(out_path, "topo_9_mr.csv"), index = False)